In [1]:
import pandas as pd
import re
import numpy as np

from IPython.display import Markdown, display

In [2]:
df = pd.read_csv('ABCDReproNimCourseStudentApplication.csv', encoding='437')
cols = {'name': 'What is your name?', 
        'email': 'What is your email address?',
       'consent': 'We (the organizers of the ABCD-ReproNim Course) will process your Personal Data provided in the following questions for the purposes of monitoring the diversity of our attendees. This is for the s...',
       'age' :'What is your age?', 
       'residence country': 'What is your Country of Residence?',
       'origin country': 'What is your Country of Origin?',
       'underrep': 'Do you identify as being a member of one of these traditionally underrepresented groups in STEM fields?',
       'ethnicity' : 'How would you describe yourself? Select all that apply to you:',
       'gender' : 'What is your current gender identity?',
       'profession' : 'What is your current profession?',
       'university affiliation status' : 'Are you currently affiliated with a university or academic institution?',
       'university' : 'What university or academic institution are you affiliated with?',
       'university country' : 'In what country is your university or academic institution located?',
       'university city' : 'In what city is your university or academic institution located?',
       'abcd affiliation status' : 'Are you affiliated with the ABCD Study¿ at an ABCD Study¿ Site (https://abcdstudy.org/study-sites/)?',
       'abcd site' : 'Which ABCD Study¿ Site are you affiliated with?',
       'discipline' : 'What is your Academic Discipline / Field of Study / Program? (e.g., Cognitive Neuroscience, Biomedical Engineering, Clinical Psychology)',
       'interests' : 'Please tell us about your interests in the ABCD-ReproNim Course. Select the top three that apply to you:',
       'py3 knowledge' : 'What is your working knowledge of Python 3 (e.g., using numpy, scipy, or similar Python 3 software libraries)?',
       'scripting knowledge' : 'What is your familiarity in scripting (e.g., writing code or using the command line) in at least one neuroimaging software package (e.g., AFNI, ANTS, FSL, FreeSurfer, SPM, etc.)?'
       }

In [3]:
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe'''
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

In [4]:
def printmd(string):
    display(Markdown(string))

In [5]:
## clean up data ##

# remove admin entries
admin_names = ['Jessica Bartley', 'Angela Laird', 'Angie Laird', 'test']
for name in admin_names:
    df.drop(df[df['Name'] == name].index, inplace = True)
    df.drop(df[df['What is your name?'] == name].index, inplace = True)

# keep rows where email (our unique identifier) is not NaN
df = df[df['What is your email address?'].notna()]

# remove empty cols
df.drop('Name', axis=1, inplace=True)
df.drop('Email', axis=1, inplace=True)

# for duplicates that have have consent and not consent listed, keep only the consent row
ids = df[cols['email']]
df_dups = df[ids.isin(ids[ids.duplicated(keep=False)])]
dup_emails = df_dups[cols['email']].tolist()

for email in dup_emails: 
    if df[df[cols['email']]==email].shape[0] == 2:
        a = list()
        index_list = list()
        drop_index_list = list()

        listOfPositions = getIndexes(df, email)

        for i in range(len(listOfPositions)):
            index = listOfPositions[i][0]
            index_list.append(index)
            consent_string = df[cols['consent']].loc[[index]].to_string()
            split_string = re.split(r'\s{2,}', consent_string)
            consent_string = split_string[1]
            a.append(consent_string)
            
        # if a duplicate indicated they both consent and do not consent then keep the row with consent
        if a[0] != a[1]:
            if 'I do not consent' in a[0]:
                drop_index = index_list[0]
                #print('email: ', email, '\nPositions: ', index_list, '\nconsent entries: ', a)
                df = df.drop(index=drop_index)
            else:
                drop_index = index_list[1]
                #print('email: ', email, '\nPositions: ', index_list, '\nconsent entries: ', a)
                df = df.drop(index=drop_index)
        else:
            pass
        
# drop the rest of the duplicates by just keeping the first entry
df = df.drop_duplicates(subset=cols['email'], keep='first')

# we changed the spelling of 'Colombian '
#df.replace({'Columbian': 'Colombian'}, regex=True)

In [6]:
df.shape

(748, 26)

In [7]:
percent_consent = df[cols['consent']].str.contains('I consent').sum() / len(df[cols['consent']])
df_consent = df[df[cols['consent']].str.contains('I consent')]
df_noconsent = df[df[cols['consent']].str.contains('I do not consent')]

# make sure number of consents and no consents match shape of origional df
if df_consent.shape[0] + df_noconsent.shape[0] == df.shape[0]:
    pass
else:
    print('there are some empty entries')


In [8]:
df_noconsent.shape

(31, 26)

In [9]:
## get numbers! ##

In [10]:
def get_mulitple_entries(list_1, list_2):
    """get elements in `list_2` that are NOT in `list_1`"""
    main_list = np.setdiff1d(list_2,list_1)
    for i in range(len(main_list)):
        if main_list[i] == 1:
            print('Get muplies function is not working')
        else:
            pass
    return main_list

def remove_df_nulls(df, col):
    df1 = df[df[col].notnull()]
    nulls = df[df[col].isnull()]
    return (df1, nulls)

def get_gender(df):
    df_gender, df_nulls = remove_df_nulls(df, cols['gender'])
    df_women = df_gender[df_gender[cols['gender']].str.contains('Woman / Trans Woman')]
    df_men = df_gender[df_gender[cols['gender']].str.contains('Man / Trans Man')]
    df_nonbinary = df_gender[df_gender[cols['gender']].str.contains('Non-Binary')]
    df_prefernoanswer = df_gender[df_gender[cols['gender']].str.contains('I prefer not to answer')]
    return (df_women, df_men, df_nonbinary, df_prefernoanswer, df_nulls)

def nonbin_breakdown(df):
    df_nonbin = df[cols['gender']]
    nonbin_men = df_nonbin[df_nonbin.str.contains('Man / Trans Man')]
    nonbin_women = df_nonbin[df_nonbin.str.contains('Woman / Trans Woman')]
    nonbin_only = df_nonbin[df_nonbin.eq('Non-Binary / Gender Queer / Gender Non-Conforming;')]
    return (nonbin_men, nonbin_women, nonbin_only)
              
def check_sums(number_list, expected_val, numbers_type):
    if sum(number_list) == expected_val:
        pass
    else:
        printmd("*warning*")
        print(numbers_type, "sums don't match!")

def print_gender_breakdown(df):
    women, men, nonbinary, prefernoanswer, blank = get_gender(df)
    print('--  women:', women.shape[0], ' men:', men.shape[0], ' nonbinary:', nonbinary.shape[0], 
          ' prefer not to answer:', prefernoanswer.shape[0], ' entry left blank:', blank.shape[0])
    nonbinary_men, nonbinary_women, nonbinary_only = nonbin_breakdown(nonbinary)
    print('---- nonbinary women:', nonbinary_women.shape[0], ' nonbinary men:', nonbinary_men.shape[0],  
          ' marked only as nonbinary:', nonbinary_only.shape[0])
    # for NIH reporting
    unknowns = [nonbinary_only.shape[0], prefernoanswer.shape[0], blank.shape[0]]
    totals_list = [men.shape[0], women.shape[0], nonbinary_only.shape[0], prefernoanswer.shape[0], blank.shape[0]]
    print('------> "Females":', women.shape[0], ' "Males":', men.shape[0], 
          ' "Sex/Gender Unknown or Nor Reported"', sum(unknowns), 
          ' "Total":', sum(totals_list), '\n')
    numbers_list = [men.shape[0], women.shape[0], nonbinary_only.shape[0], prefernoanswer.shape[0], blank.shape[0]]
    check_sums(numbers_list, sum(totals_list), 'gender')
    
def does_not_contain(df, col, string):
    new_df = df[~df[col].str.contains(string)]
    return new_df

In [11]:
# Full breakdown
printmd('**Full Ethnicity Breakdown**')
df_ethnicity = df_consent[df_consent[cols['ethnicity']].notnull()]
nulls = df_consent[df_consent[cols['ethnicity']].isnull()]

ethnicity_lables = ['Hispanic', 'American Indian', 'Asian', 'Hawaiian', 'Black', 'White', 'Middle Eastern', 'I prefer not to answer', 'Other']

# full breakdown
a_list = list()
for label in ethnicity_lables:
    df_label = df_ethnicity[df_ethnicity[cols['ethnicity']].str.contains(label)]
    print(df_label[cols['ethnicity']].shape)
    df_label.to_csv('{0}.csv'.format(label))
    print('Identify as {0}: '.format(label), df_label[cols['ethnicity']].shape[0])
    print_gender_breakdown(df_label)
    a_list.append(df_label[cols['ethnicity']].shape[0])
    if label == 'Hispanic':
        df_nothispanic = does_not_contain(df_ethnicity, cols['ethnicity'], 'Hispanic')
        print('Does not identify as {0}: '.format(label), df_nothispanic[cols['ethnicity']].shape[0], '\n')
        print_gender_breakdown(df_nothispanic)
        hisp_nothisp_list = [df_nothispanic.shape[0], df_label.shape[0]]
        check_sums(hisp_nothisp_list, df_ethnicity.shape[0], 'hispanic/non-hispanic')

print('Number of nulls among consents: ', nulls.shape[0])
print_gender_breakdown(nulls)
print('Number of no consents: ', df_noconsent.shape[0], '\n')
print_gender_breakdown(df_noconsent)

print('Totals: ', df_ethnicity.shape[0] + nulls.shape[0] + df_noconsent.shape[0])

**Full Ethnicity Breakdown**

(72,)
Identify as Hispanic:  72
--  women: 43  men: 28  nonbinary: 0  prefer not to answer: 0  entry left blank: 1
---- nonbinary women: 0  nonbinary men: 0  marked only as nonbinary: 0
------> "Females": 43  "Males": 28  "Sex/Gender Unknown or Nor Reported" 1  "Total": 72 

Does not identify as Hispanic:  641 

--  women: 407  men: 216  nonbinary: 13  prefer not to answer: 12  entry left blank: 0
---- nonbinary women: 5  nonbinary men: 2  marked only as nonbinary: 6
------> "Females": 407  "Males": 216  "Sex/Gender Unknown or Nor Reported" 18  "Total": 641 

(6,)
Identify as American Indian:  6
--  women: 3  men: 3  nonbinary: 1  prefer not to answer: 0  entry left blank: 0
---- nonbinary women: 0  nonbinary men: 1  marked only as nonbinary: 0
------> "Females": 3  "Males": 3  "Sex/Gender Unknown or Nor Reported" 0  "Total": 6 

(237,)
Identify as Asian:  237
--  women: 141  men: 93  nonbinary: 5  prefer not to answer: 1  entry left blank: 0
---- nonbinary women: 2  nonbinary men: 1  

In [12]:
# Part A
printmd('**Part A: Number of Trainees Appointed by Ethnicity and Race**')

df_ethnicity = df_consent[df_consent[cols['ethnicity']].notnull()]
nulls = df_consent[df_consent[cols['ethnicity']].isnull()]
n_nulls = nulls.shape[0]
#df_ethnicity.to_csv('test.csv')

df_hispanic = df_ethnicity[df_ethnicity[cols['ethnicity']].str.contains('Hispanic')]
df_nothispanic = does_not_contain(df_ethnicity, cols['ethnicity'], 'Hispanic')

lables = ['Hispanic', 'American Indian', 'Asian', 'Hawaiian', 'Black', 'White', 'Middle Eastern', 'I prefer not to answer', 'Other']

no_ethnicity_list = ['I prefer not to answer;']
single_ethnicity_list = ['American Indian or Alaska Native╤For example, Navajo Nation, Blackfeet Tribe, Mayan, Aztec, Native Village of Barrow Inupiat Traditional Government, Nome Eskimo Community;', 
                         'Asian╤For example, Chinese, Filipino, Asian Indian, Vietnamese, Korean, Japanese;', 
                         'Native Hawaiian or Other Pacific Islander╤For example, Native Hawaiian, Samoan, Chamorro, Tongan, Fijian, Marshallese;', 
                         'Black or African American╤For example, Jamaican, Haitian, Nigerian, Ethiopian, Somalian;', 
                         'White╤For example, German, Irish, English, Italian, Polish, French;', 
                         'Hispanic, Latino or Spanish Origin╤For example, Mexican or Mexican American, Puerto Rican, Cuban, Salvadoran, Dominican, Columbian;', 
                         'Hispanic, Latino or Spanish Origin╤For example, Mexican or Mexican American, Puerto Rican, Cuban, Salvadoran, Dominican, Colombian;',
                         'Middle Eastern or North African╤For example, Lebanese, Iranian, Egyptian, Syrian, Moroccan, Algerian;', 
                         'Other', 'I prefer not to answer;']
all_ethnicities_entered = df_ethnicity[cols['ethnicity']].tolist()

# list of all "more than one" race entries
multiple_ethnicities_list = get_mulitple_entries(single_ethnicity_list, all_ethnicities_entered)
multiple_ethnicities_list = multiple_ethnicities_list.tolist()


# here's the deal.... NIH does not consider "hispanic" as a race.
# we collected combined information on race/ethnicity
# ...so some of our entries are what the nih considers race + "hispanic"
# so what we have to do is take the multiple entries that have "hispanic" in them
# and remove the "hispanic" part and then the (what were) "multiple" entries turn into "single" entries 
# that then get summed along with the true single entires
# so lets make that happen...
print(len(multiple_ethnicities_list))
#print(multiple_ethnicities_list)
multiple_ethnicities_list_new = list()
for i in range(len(multiple_ethnicities_list)):
#    print(i)
    aslist = multiple_ethnicities_list[i].split(';')
    aslist.pop()
    asstring = ''.join(aslist)
    if 'Hispanic' in asstring:
        print('_____')
        print(aslist)
        print('+++++')
        newlist = [x for x in aslist if not 'Hispanic' in x]
        print('length', len(newlist))
        print(newlist)        
        multiple_ethnicities_list_new.append(newlist)
#        print(df_ethnicity[cols['ethnicity']])
    else:
        pass

print(len(multiple_ethnicities_list_new))
print(multiple_ethnicities_list_new)
print('number of types of multiethnicities entries: ', len(multiple_ethnicities_list_new))

#df_new = df_ethnicity[df_ethnicity[cols['ethnicity']].str.contains('Hispanic')].loc[[9, 11]]
#new = df_ethnicity[df_ethnicity[cols['ethnicity']].str.contains('Hispanic')].loc[[9, 11]][cols['ethnicity']].to_string
#df_new

**Part A: Number of Trainees Appointed by Ethnicity and Race**

13
_____
['American Indian or Alaska Native╤For example, Navajo Nation, Blackfeet Tribe, Mayan, Aztec, Native Village of Barrow Inupiat Traditional Government, Nome Eskimo Community', 'Hispanic, Latino or Spanish Origin╤For example, Mexican or Mexican American, Puerto Rican, Cuban, Salvadoran, Dominican, Colombian']
+++++
length 1
['American Indian or Alaska Native╤For example, Navajo Nation, Blackfeet Tribe, Mayan, Aztec, Native Village of Barrow Inupiat Traditional Government, Nome Eskimo Community']
_____
['Asian╤For example, Chinese, Filipino, Asian Indian, Vietnamese, Korean, Japanese', 'Hispanic, Latino or Spanish Origin╤For example, Mexican or Mexican American, Puerto Rican, Cuban, Salvadoran, Dominican, Colombian', 'Native Hawaiian or Other Pacific Islander╤For example, Native Hawaiian, Samoan, Chamorro, Tongan, Fijian, Marshallese']
+++++
length 2
['Asian╤For example, Chinese, Filipino, Asian Indian, Vietnamese, Korean, Japanese', 'Native Hawaiian or Other Pacific Islander╤For

In [13]:
#get df of single ethnicity entries
for j, single_entry in enumerate(single_ethnicity_list):
#    print(single_entry)
    ethnicity_index_list = list()
    positionTuple = getIndexes(df_ethnicity, single_entry)
    first_tuple_list = positionTuple
    for iteration, i in enumerate(range(len(positionTuple))):
        index = positionTuple[i][0]
        #print(index)
        ethnicity_index_list.append(index)
        long_string = df_ethnicity[cols['ethnicity']].loc[[index]].to_string()
        split_string = re.split(r'\s{2,}', long_string)
        string = split_string[1]
    df_single_entry = df_ethnicity.loc[ethnicity_index_list]
    print('n entries containing ONLY "{0}"'.format(single_entry), len(positionTuple))
    if 'Hispanic' in single_entry:
        printmd('NOTE: For part A we count hispanic listed alone as un unknown or not reported race. Why? Because NIH does not list hispanic as a race.')
        printmd('ALSO NOTE: There are two hispanic entries here because we changed the spelling of Colombian midway.')
    elif 'prefer not to answer' in single_entry:
        print("______________________")
        df_single_entry = df_single_entry[df_single_entry[cols['ethnicity']].str.contains('I prefer not to answer')]
    else:
        pass
    df_single_entry.to_csv('test_{0}.csv'.format(j))
    print_gender_breakdown(df_single_entry)

n entries containing ONLY "American Indian or Alaska Native╤For example, Navajo Nation, Blackfeet Tribe, Mayan, Aztec, Native Village of Barrow Inupiat Traditional Government, Nome Eskimo Community;" 2
--  women: 1  men: 1  nonbinary: 0  prefer not to answer: 0  entry left blank: 0
---- nonbinary women: 0  nonbinary men: 0  marked only as nonbinary: 0
------> "Females": 1  "Males": 1  "Sex/Gender Unknown or Nor Reported" 0  "Total": 2 

n entries containing ONLY "Asian╤For example, Chinese, Filipino, Asian Indian, Vietnamese, Korean, Japanese;" 225
--  women: 133  men: 89  nonbinary: 5  prefer not to answer: 1  entry left blank: 0
---- nonbinary women: 2  nonbinary men: 1  marked only as nonbinary: 2
------> "Females": 133  "Males": 89  "Sex/Gender Unknown or Nor Reported" 3  "Total": 225 

n entries containing ONLY "Native Hawaiian or Other Pacific Islander╤For example, Native Hawaiian, Samoan, Chamorro, Tongan, Fijian, Marshallese;" 0
--  women: 0  men: 0  nonbinary: 0  prefer not to

NOTE: For part A we count hispanic listed alone as un unknown or not reported race. Why? Because NIH does not list hispanic as a race.

ALSO NOTE: There are two hispanic entries here because we changed the spelling of Colombian midway.

--  women: 10  men: 6  nonbinary: 0  prefer not to answer: 0  entry left blank: 0
---- nonbinary women: 0  nonbinary men: 0  marked only as nonbinary: 0
------> "Females": 10  "Males": 6  "Sex/Gender Unknown or Nor Reported" 0  "Total": 16 

n entries containing ONLY "Hispanic, Latino or Spanish Origin╤For example, Mexican or Mexican American, Puerto Rican, Cuban, Salvadoran, Dominican, Colombian;" 37


NOTE: For part A we count hispanic listed alone as un unknown or not reported race. Why? Because NIH does not list hispanic as a race.

ALSO NOTE: There are two hispanic entries here because we changed the spelling of Colombian midway.

--  women: 20  men: 16  nonbinary: 0  prefer not to answer: 0  entry left blank: 1
---- nonbinary women: 0  nonbinary men: 0  marked only as nonbinary: 0
------> "Females": 20  "Males": 16  "Sex/Gender Unknown or Nor Reported" 1  "Total": 37 

n entries containing ONLY "Middle Eastern or North African╤For example, Lebanese, Iranian, Egyptian, Syrian, Moroccan, Algerian;" 34
--  women: 16  men: 16  nonbinary: 1  prefer not to answer: 1  entry left blank: 0
---- nonbinary women: 0  nonbinary men: 0  marked only as nonbinary: 1
------> "Females": 16  "Males": 16  "Sex/Gender Unknown or Nor Reported" 2  "Total": 34 

n entries containing ONLY "Other" 0
--  women: 0  men: 0  nonbinary: 0  prefer not to answer: 0  entry left blank: 0
---- nonbinary women: 0  nonbinary men: 0  marked only as nonbinary: 0
------> "Females": 0  "Males": 0  "Sex/Gender Unknown or Nor Reported" 0  "Total": 0 

n entries containing ONLY "I prefer not to answer;" 28
______________________
--  women: 5  men: 3  nonb

In [14]:
#get df multiple ethnicity entries
for n, multi_entry in enumerate(multiple_ethnicities_list):
    multi_ethnicity_index_list = list()
    split_multi_string = multi_entry.split(';')
    split_multi_string.pop()
#    print('---', n, '---', 'str len', len(split_multi_string), split_multi_string)
    print('---', n, '---')
    getPositionTuple = getIndexes(df_ethnicity, multi_entry)
    for i in range(len(getPositionTuple)):
        index = getPositionTuple[i][0]
        multi_ethnicity_index_list.append(index)
        long_string = df_ethnicity[cols['ethnicity']].loc[[index]].to_string()
        split_string = re.split(r'\s{2,}', long_string)
#        print('___', split_string[1].split(';'))
        string = split_string[1]
    df_multi_entry = df_ethnicity.loc[multi_ethnicity_index_list]
    print('n entries containing "{0}"'.format(multi_entry), len(getPositionTuple))
    if 'Hispanic' in single_entry:
        printmd('NOTE: contains Hispanic.')
    else:
        pass
    df_multi_entry.to_csv('test_{0}.csv'.format(n))
    print_gender_breakdown(df_multi_entry)
#        print(string)
#        split_on_semicol = string.split(';')
#        print(split_on_semicol)

#multi_ethnicity_df = df_ethnicity.loc[multi_ethnicity_index_list]
#multi_ethnicity_df.to_csv('test.csv')

--- 0 ---
n entries containing "American Indian or Alaska Native╤For example, Navajo Nation, Blackfeet Tribe, Mayan, Aztec, Native Village of Barrow Inupiat Traditional Government, Nome Eskimo Community;Asian╤For example, Chinese, Filipino, Asian Indian, Vietnamese, Korean, Japanese;" 1
--  women: 0  men: 1  nonbinary: 0  prefer not to answer: 0  entry left blank: 0
---- nonbinary women: 0  nonbinary men: 0  marked only as nonbinary: 0
------> "Females": 0  "Males": 1  "Sex/Gender Unknown or Nor Reported" 0  "Total": 1 

--- 1 ---
n entries containing "American Indian or Alaska Native╤For example, Navajo Nation, Blackfeet Tribe, Mayan, Aztec, Native Village of Barrow Inupiat Traditional Government, Nome Eskimo Community;Hispanic, Latino or Spanish Origin╤For example, Mexican or Mexican American, Puerto Rican, Cuban, Salvadoran, Dominican, Colombian;" 1
--  women: 1  men: 0  nonbinary: 0  prefer not to answer: 0  entry left blank: 0
---- nonbinary women: 0  nonbinary men: 0  marked only